# 加载本地保存的模型，进行评估和再训练更高的 F1 Score

In [1]:
# 加载数据集 squad_v2
from datasets import load_dataset
dataset_v2 = load_dataset("squad_v2")

/home/ubuntu/miniconda3/envs/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 查看数据集
from datasets import ClassLabel,Sequence
import random
import pandas as pd
from IPython.display import display,HTML

def show_random_elements(dataset,num_examples=10):
    assert num_examples <= len(dataset), "没有更多数据"
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0,len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0,len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column,typ in dataset.features.items():
        if isinstance(typ,ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ,Sequence) and isinstance(typ.feature,ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    
    display(HTML(df.to_html()))

# show_random_elements(dataset_v2["train"])

In [3]:
# 预处理数据
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [4]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

## Tokenizer 进阶操作

In [5]:
max_length = 384 # 实例最大长度
doc_stride = 128 # 上下文保留长度，用于保证语义

## 使用 offsets_mapping 获取原始的 input_ids

In [6]:
pad_on_right = tokenizer.padding_side == "right"
pad_on_right

True

In [7]:
# 整合以上步骤
def prepare_train_features(examples):
    # 删除问题左侧空白字符
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # 定义 tokenizer
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # 由于一个示例可能给我们提供多个特征（如果它具有很长的上下文），我们需要一个从特征到其对应示例的映射。这个键就提供了这个映射关系。
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # 偏移映射将为我们提供从令牌到原始上下文中的字符位置的映射。这将帮助我们计算开始位置和结束位置。
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # 添加 start_positions 和 end_positions
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i] # 获取 input_ids []
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i) # 获取截取后的每一个 offsets 对应的 sequence_ids()

        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # 答案开始和结束的字符索引
            start_char_index = answers["answer_start"][0]
            end_char_index = start_char_index + len(answers["text"][0])

            # 当前文本中开始token索引
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # 当前文本中结束token索引
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # 检测答案是否在 [token_start_index,token_end_index] 中
            if not (offsets[token_start_index][0] <= start_char_index and offsets[token_end_index][1] >= end_char_index):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # 将 token_index 移动到答案两端的 token 上
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char_index:
                    token_start_index += 1
                while token_end_index > 1 and offsets[token_end_index][1] >= end_char_index:
                    token_end_index -= 1
                # 考虑边界情况
                tokenized_examples["start_positions"].append(token_start_index - 1)
                tokenized_examples["end_positions"].append(token_end_index + 1) 
    
    return tokenized_examples

# tokenized_datasets = dataset_v2.map(prepare_train_features,batched=True,remove_columns=dataset_v2["train"].column_names)
# tokenized_datasets

In [8]:
# 作用到整个数据集上
# print(dataset_v2["train"].column_names)
tokenized_datasets = dataset_v2.map(
    prepare_train_features,
    batched = True,
    remove_columns = dataset_v2["train"].column_names
)
tokenized_datasets

Map: 100%|██████████████████████████| 11873/11873 [00:02<00:00, 5167.44 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 131754
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 12134
    })
})

In [9]:
# 引入模型
model_checkpoint = "distilbert-base-uncased"
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

# 设置超参数
batch_size = 64
model_dir = f"models/{model_checkpoint}-finetuned-squad"
args = TrainingArguments(
    output_dir = model_dir,
    evaluation_strategy = "epoch",
    learning_rate = 2e-5, # ? 
    save_total_limit = 5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = 3,
    weight_decay = 0.01 # ?
)

2024-03-06 01:44:34.286501: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-06 01:44:34.316230: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 01:44:34.316262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 01:44:34.316865: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-06 01:44:34.322035: I tensorflow/core/platform/cpu_feature_guar

In [10]:
# 数据整理
from transformers import default_data_collator

data_collator = default_data_collator

In [11]:
# 实例化训练器
trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer=tokenizer
)

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.442000,1.293783
2,1.166400,1.246210
3,1.038400,1.321249


TrainOutput(global_step=6177, training_loss=1.3532981681113532, metrics={'train_runtime': 4508.7095, 'train_samples_per_second': 87.666, 'train_steps_per_second': 1.37, 'total_flos': 3.873165421863629e+16, 'train_loss': 1.3532981681113532, 'epoch': 3.0})

In [13]:
model_to_save = trainer.save_model(model_dir)

## 模型评估

In [14]:
import torch

for batch in trainer.get_eval_dataloader():
    break

batch = {k: v.to(trainer.args.device) for k,v in batch.items()}

with torch.no_grad():
    output=trainer.model(**batch)


In [15]:
output.start_logits.shape,output.end_logits.shape

(torch.Size([64, 384]), torch.Size([64, 384]))

In [16]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 49,  37,  72,  80, 157,  17,  44,   0, 147, 202, 119,  52,  22,  31,
           0, 120,   0,  94,  90,   0,   0,  56,  88, 152,  23,   0,   0, 124,
           0,   0, 143,   0,  72,   0,   0,   0,  93,   0,  93, 105,  41,  13,
          33,  55,  99,  29,  80,   0,  28,  29,   0,   0, 139,  70,  49,   0,
          48,  62,  75,  19,  16,   0,   0,  12], device='cuda:0'),
 tensor([ 49,  40,  76,  81, 157,  19,  44,   0, 153, 204, 120,  52,  26,  33,
           0, 121,   0,  97,  91,   0,   0,  56,  94, 153,  24,   0,   0, 142,
          16,  52, 149,   0,  72,   0,   0,   0,  94,  27,  94, 128,  58,  15,
          28,  56, 104,  30,  81, 143,  29,  30,   0,  80, 141,  71,  51,   0,
          49,  63,  82,  20,  17,   0,   0,  15], device='cuda:0'))

In [17]:
n_best_size = 20

In [18]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# 获取最佳的起始和结束位置的索引：
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

valid_answers = []

# 遍历起始位置和结束位置的索引组合
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index:  # 需要进一步测试以检查答案是否在上下文中
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": ""  # 我们需要找到一种方法来获取与上下文中答案对应的原始子字符串
                }
            )


In [19]:
def prepare_validation_features(examples):
    # 一些问题的左侧有很多空白，这些空白并不有用且会导致上下文截断失败（分词后的问题会占用很多空间）。
    # 因此我们移除这些左侧空白
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # 使用截断和可能的填充对我们的示例进行分词，但使用步长保留溢出的令牌。这导致一个长上下文的示例可能产生
    # 几个特征，每个特征的上下文都会稍微与前一个特征的上下文重叠。
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # 由于一个示例在上下文很长时可能会产生几个特征，我们需要一个从特征映射到其对应示例的映射。这个键就是为了这个目的。
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # 我们保留产生这个特征的示例ID，并且会存储偏移映射。
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # 获取与该示例对应的序列（以了解哪些是上下文，哪些是问题）。
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # 一个示例可以产生几个文本段，这里是包含该文本段的示例的索引。
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # 将不属于上下文的偏移映射设置为None，以便容易确定一个令牌位置是否属于上下文。
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


In [20]:
validation_features = dataset_v2["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset_v2["validation"].column_names
)

Map: 100%|██████████████████████████| 11873/11873 [00:02<00:00, 5163.43 examples/s]


In [21]:
raw_predictions = trainer.predict(validation_features)

In [22]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [23]:
max_answer_length = 30

In [24]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]

# 第一个特征来自第一个示例。对于更一般的情况，我们需要将example_id匹配到一个示例索引
context = dataset_v2["validation"][0]["context"]

# 收集最佳开始/结束逻辑的索引：
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # 不考虑超出范围的答案，原因是索引超出范围或对应于输入ID的部分不在上下文中。
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # 不考虑长度小于0或大于max_answer_length的答案。
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # 我们需要细化这个测试，以检查答案是否在上下文中
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 14.020666, 'text': 'France'},
 {'score': 9.298609, 'text': 'France.'},
 {'score': 8.040493, 'text': 'a region in France'},
 {'score': 7.4637156,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway'},
 {'score': 7.231961, 'text': 'in France'},
 {'score': 7.0140047,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark'},
 {'score': 6.4636974, 'text': 'region in France'},
 {'score': 6.157026,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland'},
 {'score': 5.6163383, 'text': 'Normandy, a region in France'},
 {'score': 3.859589,
  'text': 'French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': 3.67373,
  'text': 'in the 10th and 11th centuries gave their name to Normandy

In [25]:
dataset_v2["validation"][0]["answers"]

{'text': ['France', 'France', 'France', 'France'],
 'answer_start': [159, 159, 159, 159]}

In [26]:
import collections

examples = dataset_v2["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [27]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # 构建一个从示例到其对应特征的映射。
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # 我们需要填充的字典。
    predictions = collections.OrderedDict()

    # 日志记录。
    print(f"正在后处理 {len(examples)} 个示例的预测，这些预测分散在 {len(features)} 个特征中。")

    # 遍历所有示例！
    for example_index, example in enumerate(tqdm(examples)):
        # 这些是与当前示例关联的特征的索引。
        feature_indices = features_per_example[example_index]

        min_null_score = None # 仅在squad_v2为True时使用。
        valid_answers = []
        
        context = example["context"]
        # 遍历与当前示例关联的所有特征。
        for feature_index in feature_indices:
            # 我们获取模型对这个特征的预测。
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # 这将允许我们将logits中的某些位置映射到原始上下文中的文本跨度。
            offset_mapping = features[feature_index]["offset_mapping"]

            # 更新最小空预测。
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # 浏览所有的最佳开始和结束logits，为 `n_best_size` 个最佳选择。
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # 不考虑超出范围的答案，原因是索引超出范围或对应于输入ID的部分不在上下文中。
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # 不考虑长度小于0或大于max_answer_length的答案。
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # 在极少数情况下我们没有一个非空预测，我们创建一个假预测以避免失败。
            best_answer = {"text": "", "score": 0.0}
        
        # 选择我们的最终答案：最佳答案或空答案（仅适用于squad_v2）
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [28]:
final_predictions = postprocess_qa_predictions(dataset_v2["validation"], validation_features, raw_predictions.predictions)

正在后处理 11873 个示例的预测，这些预测分散在 12134 个特征中。


100%|███████████████████████████████████████| 11873/11873 [00:14<00:00, 837.49it/s]


In [29]:
from datasets import load_metric
metric = load_metric("squad_v2")

/tmp/ipykernel_1449796/4244615236.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2")
/home/ubuntu/miniconda3/envs/transformers/lib/python3.11/site-packages/datasets/load.py:752: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [30]:
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in dataset_v2["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 61.265055167186055,
 'f1': 64.92858615097008,
 'total': 11873,
 'HasAns_exact': 64.00134952766531,
 'HasAns_f1': 71.33891757261564,
 'HasAns_total': 5928,
 'NoAns_exact': 58.53658536585366,
 'NoAns_f1': 58.53658536585366,
 'NoAns_total': 5945,
 'best_exact': 61.29032258064516,
 'best_exact_thresh': 0.0,
 'best_f1': 64.92858615097029,
 'best_f1_thresh': 0.0}